In [ ]:
#Import all the libraries required

import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import OrdinalEncoder
pd.set_option('display.max_columns', None)

In [ ]:
#Read all the player data files for each year in FIFA

df15 = pd.read_csv('raw_data/players_15.csv',low_memory=False)
df16 = pd.read_csv('raw_data/players_16.csv',low_memory=False)
df17 = pd.read_csv('raw_data/players_17.csv',low_memory=False)
df18 = pd.read_csv('raw_data/players_18.csv',low_memory=False)
df19 = pd.read_csv('raw_data/players_19.csv',low_memory=False)
df20 = pd.read_csv('raw_data/players_20.csv',low_memory=False)
df21 = pd.read_csv('raw_data/players_21.csv',low_memory=False)
df22 = pd.read_csv('raw_data/players_22.csv',low_memory=False)

In [ ]:
#Add a column to identify the seasons in each dataframe

df15['season']=1415
df16['season']=1516
df17['season']=1617
df18['season']=1718
df19['season']=1819
df20['season']=1920
df21['season']=2021
df22['season']=2122

In [ ]:
#Concat every year's player data into one single dataframe with season information

full_df = pd.concat([df15,df16,df17,df18,df19,df20,df21,df22])
full_df.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,season
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,550000.0,27,1987-06-24,169,67,241.0,FC Barcelona,Spain Primera Division,1.0,CF,10.0,NaN,2004-07-01,2018.0,52,Argentina,1369.0,CF,10.0,Left,3,4,5,Medium/Low,Normal (170-),Yes,NaN,"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",93.0,89.0,86.0,96.0,27.0,63.0,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,NaN,25,21,20,6,11,15,14,8,NaN,89+3,89+3,89+3,92+3,90+3,90+3,90+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,1415
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,79000000.0,375000.0,29,1985-02-05,185,80,243.0,Real Madrid CF,Spain Primera Division,1.0,LW,7.0,NaN,2009-07-01,2018.0,38,Portugal,1354.0,LW,7.0,Right,4,5,5,High/Low,Normal (185+),Yes,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",93.0,93.0,81.0,91.0,32.0,79.0,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,NaN,22,31,23,7,11,15,14,11,NaN,91+1,91+1,91+1,89+3,91+1,91+1,91+1,89+3,89+3,89+3,89+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png,https://cdn.sofifa.net/teams/243/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png,1415
2,9014,https://sofifa.com/player/9014/arjen-robben/15...,A. Robben,Arjen Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,1984-01-23,180,80,21.0,FC Bayern München,German 1. Bundesliga,1.0,SUB,10.0,NaN,2009-08-28,2017.0,34,Netherlands,105035.0,RS,11.0,Left,2,4,5,High/Low,Normal (170-185),Yes,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Diver, Injury Prone, Avoids Using Weaker Foot,...",93.0,86.0,83.0,92.0,32.0,64.0,80,85,50,86,86,93,85,83,76,90,93,93,93,89,91,86,61,78,65,90,47,39,89,84,80,NaN,29,26,26,10,8,11,5,15,NaN,84+3,84+3,84+3,88+2,87+3,87+3,87+3,88+2,88+2,88+2,88+2,87+3,78+3,78+3,78+3,87+3,64+3,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/

In [ ]:
#Create mapping table for clubs and national teams

df_club_info = full_df[['club_team_id','club_name']].drop_duplicates(keep='first')
df_nation_info = full_df[['nationality_id','nationality_name']].drop_duplicates(keep='first')

In [ ]:
#Added 'nationality_name' and dropped ['nation_jersey_number', 'nation_team_id', 'club_team_id', 'nation_position', 'dob', 'club_team_id'] from pervious version

full_df = full_df.drop(['player_url','long_name','club_jersey_number','club_loaned_from','real_face','release_clause_eur',
              'player_face_url','club_logo_url','club_flag_url','nation_logo_url','nation_flag_url','club_joined', 'dob',
              'club_contract_valid_until','nationality_id','club_team_id', 'nation_team_id', 'nation_jersey_number', 'nation_position'],axis=1)

In [ ]:
#full_df.head()
full_df.info()
#full_df[full_df['nation_team_id'].isnull()].head()

#full_df[full_df['club_team_id']==21].head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142079 entries, 0 to 19238
Data columns (total 92 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   sofifa_id                    142079 non-null  int64  
 1   short_name                   142079 non-null  object 
 2   player_positions             142079 non-null  object 
 3   overall                      142079 non-null  int64  
 4   potential                    142079 non-null  int64  
 5   value_eur                    140182 non-null  float64
 6   wage_eur                     140457 non-null  float64
 7   age                          142079 non-null  int64  
 8   height_cm                    142079 non-null  int64  
 9   weight_kg                    142079 non-null  int64  
 10  club_name                    140449 non-null  object 
 11  league_name                  140449 non-null  object 
 12  league_level                 140064 non-null  float64
 13  

In [ ]:
#Save full_df, club and nation info as csv files

#full_df.to_csv('player_combined_fifa.csv')
#full_df_club_info.to_csv('club_info_fifa.csv')
#full_df_nation_info.to_csv('nation_info_fifa.csv')

**End of Tommy's part, this is where Tsz Kin started**

In [ ]:
#full_df = pd.read_csv('Cleaned_Data/player_combined_fifa.csv')

In [ ]:
df = full_df[full_df['player_positions'] != 'GK']
df.head()

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_name,league_level,club_position,nationality_name,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,season
0,158023,L. Messi,CF,93,95,100500000.0,550000.0,27,169,67,FC Barcelona,Spain Primera Division,1.0,CF,Argentina,Left,3,4,5,Medium/Low,Normal (170-),"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",93.0,89.0,86.0,96.0,27.0,63.0,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,NaN,25,21,20,6,11,15,14,8,NaN,89+3,89+3,89+3,92+3,90+3,90+3,90+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,1415
1,20801,Cristiano Ronaldo,"LW, LM",92,92,79000000.0,375000.0,29,185,80,Real Madrid CF,Spain Primera Division,1.0,LW,Portugal,Right,4,5,5,High/Low,Normal (185+),"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",93.0,93.0,81.0,91.0,32.0,79.0,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,NaN,22,31,23,7,11,15,14,11,NaN,91+1,91+1,91+1,89+3,91+1,91+1,91+1,89+3,89+3,89+3,89+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,1415
2,9014,A. Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,180,80,FC Bayern München,German 1. Bundesliga,1.0,SUB,Netherlands,Left,2,4,5,High/Low,Normal (170-185),"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Diver, Injury Prone, Avoids Using Weaker Foot,...",93.0,86.0,83.0,92.0,32.0,64.0,80,85,50,86,86,93,85,83,76,90,93,93,93,89,91,86,61,78,65,90,47,39,89,84,80,NaN,29,26,26,10,8,11,5,15,NaN,84+3,84+3,84+3,88+2,87+3,87+3,87+3,88+2,88+2,88+2,88+2,87+3,78+3,78+3,78+3,87+3,64+3,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,1415
3,41236,Z. Ibrahimović,ST,90,90,52500000.0,275000.0,32,195,95,Paris Saint-Germain,French Ligue 1,1.0,ST,Sweden,Right,4,4,5,Medium/Low,Normal (185+),"#Poacher, #Aerial Threat, #Distance Shooter, #...","Power Free-Kick, Leadership, Flair, Long Shot ...",76.0,91.0,81.0,86.0,34.0,86.0,76,91,76,84,92,88,80,80,76,90,74,77,86,85,41,93,72,78,93,88,84,20,86,83,91,NaN,25,41,27,13,15,10,9,12,NaN,87+3,87+3,87+3,84+3,86+3,86+3,86+3,84+3,86+3,86+3,86+3,83+3,76+3,76+3,76+3,83+3,61+3,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,1415
5,41,Iniesta,"CM, LW",89,89,36000000.0,250000.0,30,170,65,FC Barcelona,Spain Primera Division,1.0,LCM,Spain,Right,4,4,5,High/Medium,Normal (170-),"#Dribbler, #Playmaker","Finesse Shot, Playmaker (AI), Technical Dribbl...",75.0,72.0,89.0,91.0,59.0,63.0,85,73,54,93,74,92,80,70,89,94,76,75,83,90,86,65,54,78,59,75,58,68,87,93,71,NaN,57,57,56,6,13,6,13,7,NaN,77+3,77+3,77+3,87+2,82+3,82+3,82+3,87+2,88+1,88+1,88+1,88+1,86+3,86+3,86+3,88+1,76+3,77+3,77+3,77+3,76+3,70+3,63+3,63+3,63+3,70+3,14+3,1415


In [ ]:
#Sample data for Messi

df[df['sofifa_id'] == 158023]

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_name,league_level,club_position,nationality_name,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,season
0,158023,L. Messi,CF,93,95,100500000.0,550000.0,27,169,67,FC Barcelona,Spain Primera Division,1.0,CF,Argentina,Left,3,4,5,Medium/Low,Normal (170-),"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",93.0,89.0,86.0,96.0,27.0,63.0,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,NaN,25,21,20,6,11,15,14,8,NaN,89+3,89+3,89+3,92+3,90+3,90+3,90+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,1415
0,158023,L. Messi,"RW, CF",94,95,111000000.0,550000.0,28,170,72,FC Barcelona,Spain Primera Division,1.0,RW,Argentina,Left,4,4,5,Medium/Low,Unique,"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",92.0,88.0,86.0,95.0,24.0,62.0,80,93,71,88,85,96,89,90,79,96,95,90,92,92,95,80,68,76,59,88,48,22,90,90,74,NaN,13,23,21,6,11,15,14,8,NaN,87+3,87+3,87+3,91+3,91+3,91+3,91+3,91+3,91+3,91+3,91+3,90+3,82+3,82+3,82+3,90+3,62+3,57+3,57+3,57+3,62+3,57+3,44+3,44+3,44+3,57+3,19+3,1516
1,158023,L. Messi,RW,93,93,89000000.0,575000.0,29,170,72,FC Barcelona,Spain Primera Division,1.0,RW,Argentina,Left,4,4,5,Medium/Medium,Unique,"#Dribbler, #FK Specialist, #Acrobat, #Clinical...","Finesse Shot, Long Shot Taker (AI), Speed Drib...",89.0,90.0,86.0,96.0,26.0,61.0,77,95,71,88,85,97,89,90,87,95,92,87,90,95,95,83,68,74,59,88,48,22,93,90,74,94.0,13,28,26,6,11,15,14,8,NaN,88+4,88+4,88+4,91+2,92+1,92+1,92+1,91+2,92+1,92+1,92+1,90+3,84+4,84+4,84+4,90+3,62+4,59+4,59+4,59+4,62+4,57+4,45+4,45+4,45+4,57+4,19+4,1617
1,158023,L. Messi,RW,93,93,105000000.0,575000.0,30,170,72,FC Barcelona,Spain Primera Division,1.0,RW,Argentina,Left,4,4,5,Medium/Medium,Unique,"#Dribbler, #FK Specialist, #Acrobat, #Clinical...","Finesse Shot, Long Shot Taker (AI), Speed Drib...",89.0,90.0,86.0,96.0,26.0,61.0,77,95,71,88,85,97,89,90,87,95,92,87,90,95,95,85,68,73,59,88,48,22,93,90,74,96.0,13,28,26,6,11,15,14,8,NaN,88+4,88+4,88+4,91+2,92+1,92+1,92+1,91+2,92+1,92+1,92+1,90+3,84+4,84+4,84+4,90+3,62+4,59+4,59+4,59+4,62+4,57+4,45+4,45+4,45+4,57+4,19+4,1718
1,158023,L. Messi,"CF, RW, ST",94,94,110500000.0,575000.0,31,170,72,FC Barcelona,Spain Primera Division,1.0,RW,Argentina,Left,4,4,5,Medium/Medium,Unique,"#Dribbler, #Distance Shooter, #FK Specialist, ...","Finesse Shot, Long Shot Taker (AI), Speed Drib...",88.0,91.0,88.0,96.0,32.0,61.0,77,95,70,90,86,97,93,94,87,96,91,86,91,95,95,85,68,72,59,94,48,22,94,94,75,96.0,33,28,26,6,11,15,14,8,NaN,89+5,89+5,89+5,92+2,93+1,93+1,93+1,92+2,93+1,93+1,93+1,91+3,85+7,85+7,85+7,91+3,63+7,61+7,61+7,61+7,63+7,59+7,48+7,48+7,48+7,59+7,19+7,1819
0,158023,L. Messi,"RW, CF, ST",94,94,95500000.0,560000.0,32,170,72,FC Barcelona,Spain Primera Division,1.0,RW,Argentina,Left,4,4,5,Medium/Low,Unique,"#Dribbler, #Distance Shooter, #Crosser, #FK Sp...","Finesse Shot, Long Shot Taker (AI), Speed Drib...",87.

In [ ]:
df.columns

Index(['sofifa_id', 'short_name', 'player_positions', 'overall', 'potential',
       'value_eur', 'wage_eur', 'age', 'height_cm', 'weight_kg', 'club_name',
       'league_name', 'league_level', 'club_position', 'nationality_name',
       'preferred_foot', 'weak_foot', 'skill_moves',
       'international_reputation', 'work_rate', 'body_type', 'player_tags',
       'player_traits', 'pace', 'shooting', 'passing', 'dribbling',
       'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentalit

In [ ]:
df[df['sofifa_id'] == 158023][['short_name','mentality_composure','season']]

,short_name,mentality_composure,season
0,L. Messi,NaN,1415
0,L. Messi,NaN,1516
1,L. Messi,94.0,1617
1,L. Messi,96.0,1718
1,L. Messi,96.0,1819
0,L. Messi,96.0,1920
0,L. Messi,96.0,2021
0,L. Messi,96.0,2122


In [ ]:
#Changed from "apply" to "transform"

#Original by Tsz Kin
#df['mentality_composure'] = df.groupby('sofifa_id')['mentality_composure'].apply(lambda x: x.bfill())

#Method 1: Use bfill
#df['mentality_composure'] = df.groupby('sofifa_id')['mentality_composure'].transform(lambda x: x.bfill())

#Method 2: Use the average difference between overall and mentality_composure of other players
df['diff'] = df.overall - df['mentality_composure']
df['mentality_composure'] = df['mentality_composure'].fillna(df['overall']-round(df['diff'].mean()))

df[df['sofifa_id'] == 158023][['short_name','mentality_composure','season']]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,short_name,mentality_composure,season
0,L. Messi,87.0,1415
0,L. Messi,88.0,1516
1,L. Messi,94.0,1617
1,L. Messi,96.0,1718
1,L. Messi,96.0,1819
0,L. Messi,96.0,1920
0,L. Messi,96.0,2021
0,L. Messi,96.0,2122


In [ ]:
df.head()

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_name,league_level,club_position,nationality_name,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,season,diff
0,158023,L. Messi,CF,93,95,100500000.0,550000.0,27,169,67,FC Barcelona,Spain Primera Division,1.0,CF,Argentina,Left,3,4,5,Medium/Low,Normal (170-),"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",93.0,89.0,86.0,96.0,27.0,63.0,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,87.0,25,21,20,6,11,15,14,8,NaN,89+3,89+3,89+3,92+3,90+3,90+3,90+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,1415,NaN
1,20801,Cristiano Ronaldo,"LW, LM",92,92,79000000.0,375000.0,29,185,80,Real Madrid CF,Spain Primera Division,1.0,LW,Portugal,Right,4,5,5,High/Low,Normal (185+),"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",93.0,93.0,81.0,91.0,32.0,79.0,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,86.0,22,31,23,7,11,15,14,11,NaN,91+1,91+1,91+1,89+3,91+1,91+1,91+1,89+3,89+3,89+3,89+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,1415,NaN
2,9014,A. Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,180,80,FC Bayern München,German 1. Bundesliga,1.0,SUB,Netherlands,Left,2,4,5,High/Low,Normal (170-185),"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Diver, Injury Prone, Avoids Using Weaker Foot,...",93.0,86.0,83.0,92.0,32.0,64.0,80,85,50,86,86,93,85,83,76,90,93,93,93,89,91,86,61,78,65,90,47,39,89,84,80,84.0,29,26,26,10,8,11,5,15,NaN,84+3,84+3,84+3,88+2,87+3,87+3,87+3,88+2,88+2,88+2,88+2,87+3,78+3,78+3,78+3,87+3,64+3,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,1415,NaN
3,41236,Z. Ibrahimović,ST,90,90,52500000.0,275000.0,32,195,95,Paris Saint-Germain,French Ligue 1,1.0,ST,Sweden,Right,4,4,5,Medium/Low,Normal (185+),"#Poacher, #Aerial Threat, #Distance Shooter, #...","Power Free-Kick, Leadership, Flair, Long Shot ...",76.0,91.0,81.0,86.0,34.0,86.0,76,91,76,84,92,88,80,80,76,90,74,77,86,85,41,93,72,78,93,88,84,20,86,83,91,84.0,25,41,27,13,15,10,9,12,NaN,87+3,87+3,87+3,84+3,86+3,86+3,86+3,84+3,86+3,86+3,86+3,83+3,76+3,76+3,76+3,83+3,61+3,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,1415,NaN
5,41,Iniesta,"CM, LW",89,89,36000000.0,250000.0,30,170,65,FC Barcelona,Spain Primera Division,1.0,LCM,Spain,Right,4,4,5,High/Medium,Normal (170-),"#Dribbler, #Playmaker","Finesse Shot, Playmaker (AI), Technical Dribbl...",75.0,72.0,89.0,91.0,59.0,63.0,85,73,54,93,74,92,80,70,89,94,76,75,83,90,86,65,54,78,59,75,58,68,87,93,71,83.0,57,57,56,6,13,6,13,7,NaN,77+3,77+3,77+3,87+2,82+3,82+3,82+3,87+2,88+1,88+1,88+1,88+1,86+3,86+3,86+3,88+1,76+3,77+3,77+3,77+3,76+3,70+3,63+3,63+3,63+3,70+3,14+3,1415,NaN


In [ ]:
#df[['Unnamed: 0', 'sofifa_id', 'short_name', 'player_positions', 'overall',
#       'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm',
#       'weight_kg', 'club_team_id', 'league_name', 'league_level',
#       'club_position', 'nation_team_id', 'nation_position',
#       'nation_jersey_number', 'preferred_foot', 'weak_foot', 'skill_moves',
#       'international_reputation', 'work_rate', 'body_type', 'player_tags',
#       'player_traits', 'pace', 'shooting', 'passing', 'dribbling',
#       'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
#       'attacking_heading_accuracy', 'attacking_short_passing',
#       'attacking_volleys', 'skill_dribbling', 'skill_curve',
#       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
#       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
#       'movement_reactions', 'movement_balance', 'power_shot_power',
#       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
#       'mentality_aggression', 'mentality_interceptions',
#       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
#       'mentality_composure', 'defending_marking_awareness',
#       'defending_standing_tackle', 'defending_sliding_tackle',
#       'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
#       'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed',
#       'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
#       'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb',
#       'lcb', 'cb', 'rcb', 'rb', 'gk', 'season']]

df[['sofifa_id', 'short_name', 'player_positions', 'overall',
       'potential', 'value_eur', 'wage_eur', 'age', 'height_cm',
       'weight_kg', 'league_name', 'league_level', 'nationality_name',
       'club_position', 'preferred_foot', 'weak_foot', 'skill_moves',
       'international_reputation', 'work_rate', 'body_type', 'player_tags',
       'player_traits', 'pace', 'shooting', 'passing', 'dribbling',
       'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking_awareness',
       'defending_standing_tackle', 'defending_sliding_tackle',
       'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
       'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed',
       'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
       'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb',
       'lcb', 'cb', 'rcb', 'rb', 'gk', 'season']].head()

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_name,league_level,nationality_name,club_position,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,season
0,158023,L. Messi,CF,93,95,100500000.0,550000.0,27,169,67,Spain Primera Division,1.0,Argentina,CF,Left,3,4,5,Medium/Low,Normal (170-),"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",93.0,89.0,86.0,96.0,27.0,63.0,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,87.0,25,21,20,6,11,15,14,8,NaN,89+3,89+3,89+3,92+3,90+3,90+3,90+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,1415
1,20801,Cristiano Ronaldo,"LW, LM",92,92,79000000.0,375000.0,29,185,80,Spain Primera Division,1.0,Portugal,LW,Right,4,5,5,High/Low,Normal (185+),"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",93.0,93.0,81.0,91.0,32.0,79.0,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,86.0,22,31,23,7,11,15,14,11,NaN,91+1,91+1,91+1,89+3,91+1,91+1,91+1,89+3,89+3,89+3,89+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,1415
2,9014,A. Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,180,80,German 1. Bundesliga,1.0,Netherlands,SUB,Left,2,4,5,High/Low,Normal (170-185),"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Diver, Injury Prone, Avoids Using Weaker Foot,...",93.0,86.0,83.0,92.0,32.0,64.0,80,85,50,86,86,93,85,83,76,90,93,93,93,89,91,86,61,78,65,90,47,39,89,84,80,84.0,29,26,26,10,8,11,5,15,NaN,84+3,84+3,84+3,88+2,87+3,87+3,87+3,88+2,88+2,88+2,88+2,87+3,78+3,78+3,78+3,87+3,64+3,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,1415
3,41236,Z. Ibrahimović,ST,90,90,52500000.0,275000.0,32,195,95,French Ligue 1,1.0,Sweden,ST,Right,4,4,5,Medium/Low,Normal (185+),"#Poacher, #Aerial Threat, #Distance Shooter, #...","Power Free-Kick, Leadership, Flair, Long Shot ...",76.0,91.0,81.0,86.0,34.0,86.0,76,91,76,84,92,88,80,80,76,90,74,77,86,85,41,93,72,78,93,88,84,20,86,83,91,84.0,25,41,27,13,15,10,9,12,NaN,87+3,87+3,87+3,84+3,86+3,86+3,86+3,84+3,86+3,86+3,86+3,83+3,76+3,76+3,76+3,83+3,61+3,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,1415
5,41,Iniesta,"CM, LW",89,89,36000000.0,250000.0,30,170,65,Spain Primera Division,1.0,Spain,LCM,Right,4,4,5,High/Medium,Normal (170-),"#Dribbler, #Playmaker","Finesse Shot, Playmaker (AI), Technical Dribbl...",75.0,72.0,89.0,91.0,59.0,63.0,85,73,54,93,74,92,80,70,89,94,76,75,83,90,86,65,54,78,59,75,58,68,87,93,71,83.0,57,57,56,6,13,6,13,7,NaN,77+3,77+3,77+3,87+2,82+3,82+3,82+3,87+2,88+1,88+1,88+1,88+1,86+3,86+3,86+3,88+1,76+3,77+3,77+3,77+3,76+3,70+3,63+3,63+3,63+3,70+3,14+3,1415


In [ ]:
df[['work_rate', 'body_type', 'player_tags', 'player_traits']]

,work_rate,body_type,player_tags,player_traits
0,Medium/Low,Normal (170-),"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl..."
1,High/Low,Normal (185+),"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Power Free-Kick, Flair, Long Shot Taker (AI), ..."
2,High/Low,Normal (170-185),"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Diver, Injury Prone, Avoids Using Weaker Foot,..."
3,Medium/Low,Normal (185+),"#Poacher, #Aerial Threat, #Distance Shooter, #...","Power Free-Kick, Leadership, Flair, Long Shot ..."
5,High/Medium,Normal (170-),"#Dribbler, #Playmaker","Finesse Shot, Playmaker (AI), Technical Dribbl..."
...,...,...,...,...
19234,Medium/Medium,Lean (170-185),NaN,NaN
19235,Medium/Medium,Lean (170-185),NaN,NaN
19236,Medium/Medium,Lean (170-185),NaN,NaN
19237,Medium/Medium,Lean (170-185),NaN,NaN


In [ ]:
#Remove (AI) from the player_traits

df.player_traits.str.replace(' \(AI\)', '', regex=True)

0        Finesse Shot, Speed Dribbler, One Club Player,...
1        Power Free-Kick, Flair, Long Shot Taker, Speed...
2        Diver, Injury Prone, Avoids Using Weaker Foot,...
3        Power Free-Kick, Leadership, Flair, Long Shot ...
5              Finesse Shot, Playmaker, Technical Dribbler
                               ...                        
19234                                                  NaN
19235                                                  NaN
19236                                                  NaN
19237                                                  NaN
19238                                                  NaN
Name: player_traits, Length: 126288, dtype: object

In [ ]:
#For each player_traits and player_tags, a binary feature is created

l = df.player_traits.str.replace(' \(AI\)', '', regex=True).dropna()
traits_ls = list(set([item.strip() for sublist in l for item in sublist.split(",")]))

l = df.player_tags.str.replace('#', '').dropna()
tag_ls = list(set([item.strip() for sublist in l for item in sublist.split(",")]))

df['player_traits'].fillna('Null',inplace=True)
for trait in traits_ls:
    df.loc[df['player_traits'].str.contains(trait), trait] = 1
    df[trait].fillna(0,inplace=True)
df.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_name,league_level,club_position,nationality_name,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,season,diff,One Club Player,Avoids Using Weaker Foot,Playmaker,Dives Into Tackles,Finesse Shot,Power Free-Kick,Leadership,Power Header,Technical Dribbler,Early Crosser,Takes Finesse Free Kicks,Through Ball,Giant Throw-in,Beat Offside Trap,Outside Foot Shot,Long Passer,Set Play Specialist,Chip Shot,Diver,Team Player,Injury Free,Injury Prone,Swerve Pass,Solid Player,Selfish,Speed Dribbler,Flair,Long Shot Taker,Long Throw-in,Backs Into Player,Target Forward
0,158023,L. Messi,CF,93,95,100500000.0,550000.0,27,169,67,FC Barcelona,Spain Primera Division,1.0,CF,Argentina,Left,3,4,5,Medium/Low,Normal (170-),"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",93.0,89.0,86.0,96.0,27.0,63.0,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,87.0,25,21,20,6,11,15,14,8,NaN,89+3,89+3,89+3,92+3,90+3,90+3,90+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,1415,NaN,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,20801,Cristiano Ronaldo,"LW, LM",92,92,79000000.0,375000.0,29,185,80,Real Madrid CF,Spain Primera Division,1.0,LW,Portugal,Right,4,5,5,High/Low,Normal (185+),"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",93.0,93.0,81.0,91.0,32.0,79.0,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,86.0,22,31,23,7,11,15,14,11,NaN,91+1,91+1,91+1,89+3,91+1,91+1,91+1,89+3,89+3,89+3,89+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,1415,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
2,9014,A. Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,180,80,FC Bayern München,German 1. Bundesliga,1.0,SUB,Netherlands,Left,2,4,5,High/Low,Normal (170-185),"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Diver, Injury Prone, Avoids Using Weaker Foot,...",93.0,86.0,83.0,92.0,32.0,64.0,80,85,50,86,86,93,85,83,76,90,93,93,93,89,91,86,61,78,65,90,47,39,89,84,80,84.0,29,26,26,10,8,11,5,15,NaN,84+3,84+3,84+3,88+2,87+3,87+3,87+3,88+2,88+2,88+2,88+2,87+3,78+3,78+3,78+3,87+3,64+3,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,1415,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
3,41236,Z. Ibrahimović,ST,90,90,52500000.0,275000.0,32,195,95,Paris Saint-Germain,French Ligue 1,1.0,ST,Sweden,Right,4,4,5,Medium/Low,Normal (185+),"#Poacher, #Aerial Threat, #Distance Shooter, #...","Power Free-Kick, Leadership, Flair, Long Shot ...",76.0,91.0,81.0,86.0,34.0,86.0,76,91,76,84,92,88,80,80,76,90,74,77,86,85,41,93,72,78,93,88,84,20,86,83,91,84.0,25,41,27,13,15,10,9,12,NaN,87+3,87+3,87+3,84+3,86+3,86+3,86+3,84+3,86+3,86+3,

In [ ]:
df['player_tags'].fillna('Null',inplace=True)
for tag in tag_ls:
    df.loc[df['player_tags'].str.contains(tag), tag] = 1
    df[tag].fillna(0,inplace=True)
    
df.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_name,league_level,club_position,nationality_name,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,season,diff,One Club Player,Avoids Using Weaker Foot,Playmaker,Dives Into Tackles,Finesse Shot,Power Free-Kick,Leadership,Power Header,Technical Dribbler,Early Crosser,Takes Finesse Free Kicks,Through Ball,Giant Throw-in,Beat Offside Trap,Outside Foot Shot,Long Passer,Set Play Specialist,Chip Shot,Diver,Team Player,Injury Free,Injury Prone,Swerve Pass,Solid Player,Selfish,Speed Dribbler,Flair,Long Shot Taker,Long Throw-in,Backs Into Player,Target Forward,Speedster,Complete Defender,Dribbler,Tackling,Acrobat,Poacher,Crosser,FK Specialist,Complete Forward,Complete Midfielder,Engine,Clinical Finisher,Aerial Threat,Tactician,Distance Shooter,Strength
0,158023,L. Messi,CF,93,95,100500000.0,550000.0,27,169,67,FC Barcelona,Spain Primera Division,1.0,CF,Argentina,Left,3,4,5,Medium/Low,Normal (170-),"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",93.0,89.0,86.0,96.0,27.0,63.0,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,87.0,25,21,20,6,11,15,14,8,NaN,89+3,89+3,89+3,92+3,90+3,90+3,90+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,1415,NaN,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,20801,Cristiano Ronaldo,"LW, LM",92,92,79000000.0,375000.0,29,185,80,Real Madrid CF,Spain Primera Division,1.0,LW,Portugal,Right,4,5,5,High/Low,Normal (185+),"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",93.0,93.0,81.0,91.0,32.0,79.0,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,86.0,22,31,23,7,11,15,14,11,NaN,91+1,91+1,91+1,89+3,91+1,91+1,91+1,89+3,89+3,89+3,89+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,1415,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,9014,A. Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,180,80,FC Bayern München,German 1. Bundesliga,1.0,SUB,Netherlands,Left,2,4,5,High/Low,Normal (170-185),"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Diver, Injury Prone, Avoids Using Weaker Foot,...",93.0,86.0,83.0,92.0,32.0,64.0,80,85,50,86,86,93,85,83,76,90,93,93,93,89,91,86,61,78,65,90,47,39,89,84,80,84.0,29,26,26,10,8,11,5,15,NaN,84+3,84+3,84+3,88+2,87+3,87+3,87+3,88+2,88+2,88+2,88+2,87+3,78+3,78+3,78+3,87+3,64+3,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,1415,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,41236,Z. Ibrahimović,ST,90,90,52500000.0,275000.0

In [ ]:
#Add up the rating for each position for each player

eval_ls = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
       'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb',
       'lcb', 'cb', 'rcb', 'rb', 'gk']

for pos in eval_ls:
    df[pos] = df[pos].apply(lambda x: eval(x))
df[eval_ls]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,92,92,92,95,93,93,93,95,95,95,95,93,82,82,82,93,65,65,65,65,65,57,48,48,48,57,18
1,92,92,92,92,92,92,92,92,92,92,92,90,80,80,80,90,66,66,66,66,66,60,55,55,55,60,19
2,87,87,87,90,90,90,90,90,90,90,90,90,81,81,81,90,67,67,67,67,67,58,49,49,49,58,17
3,90,90,90,87,89,89,89,87,89,89,89,86,79,79,79,86,64,68,68,68,64,59,58,58,58,59,20
5,80,80,80,89,85,85,85,89,89,89,89,89,89,89,89,89,79,80,80,80,79,73,66,66,66,73,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,46,46,46,46,45,45,45,46,48,48,48,49,48,48,48,49,50,48,48,48,50,50,48,48,48,50,17
19235,47,47,47,47,46,46,46,47,49,49,49,51,49,49,49,51,51,48,48,48,51,50,46,46,46,50,16
19236,47,47,47,46,46,46,46,46,49,49,49,50,49,49,49,50,49,48,48,48,49,49,47,47,47,49,14
19237,49,49,49,47,47,47,47,47,47,47,47,47,41,41,41,47,35,31,31,31,35,34,28,28,28,34,17


In [ ]:
df[['work_rate', 'body_type','preferred_foot','weak_foot']]

,work_rate,body_type,preferred_foot,weak_foot
0,Medium/Low,Normal (170-),Left,3
1,High/Low,Normal (185+),Right,4
2,High/Low,Normal (170-185),Left,2
3,Medium/Low,Normal (185+),Right,4
5,High/Medium,Normal (170-),Right,4
...,...,...,...,...
19234,Medium/Medium,Lean (170-185),Right,3
19235,Medium/Medium,Lean (170-185),Right,3
19236,Medium/Medium,Lean (170-185),Right,3
19237,Medium/Medium,Lean (170-185),Right,3


In [ ]:
#Split the workrate column (which has both attacking and defending workrates) into 2 columns

df[['att_workrate','def_workrate']] = df.work_rate.str.split("/",expand=True)

enc = OrdinalEncoder(categories=[['Low','Medium','High']])
df['att_workrate'] = enc.fit_transform(df[['att_workrate']])
df['def_workrate'] = enc.transform(df[['def_workrate']])
df.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- def_workrate
Feature names seen at fit time,

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_name,league_level,club_position,nationality_name,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,season,diff,One Club Player,Avoids Using Weaker Foot,Playmaker,Dives Into Tackles,Finesse Shot,Power Free-Kick,Leadership,Power Header,Technical Dribbler,Early Crosser,Takes Finesse Free Kicks,Through Ball,Giant Throw-in,Beat Offside Trap,Outside Foot Shot,Long Passer,Set Play Specialist,Chip Shot,Diver,Team Player,Injury Free,Injury Prone,Swerve Pass,Solid Player,Selfish,Speed Dribbler,Flair,Long Shot Taker,Long Throw-in,Backs Into Player,Target Forward,Speedster,Complete Defender,Dribbler,Tackling,Acrobat,Poacher,Crosser,FK Specialist,Complete Forward,Complete Midfielder,Engine,Clinical Finisher,Aerial Threat,Tactician,Distance Shooter,Strength,att_workrate,def_workrate
0,158023,L. Messi,CF,93,95,100500000.0,550000.0,27,169,67,FC Barcelona,Spain Primera Division,1.0,CF,Argentina,Left,3,4,5,Medium/Low,Normal (170-),"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",93.0,89.0,86.0,96.0,27.0,63.0,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,87.0,25,21,20,6,11,15,14,8,NaN,92,92,92,95,93,93,93,95,95,95,95,93,82,82,82,93,65,65,65,65,65,57,48,48,48,57,18,1415,NaN,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,20801,Cristiano Ronaldo,"LW, LM",92,92,79000000.0,375000.0,29,185,80,Real Madrid CF,Spain Primera Division,1.0,LW,Portugal,Right,4,5,5,High/Low,Normal (185+),"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",93.0,93.0,81.0,91.0,32.0,79.0,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,86.0,22,31,23,7,11,15,14,11,NaN,92,92,92,92,92,92,92,92,92,92,92,90,80,80,80,90,66,66,66,66,66,60,55,55,55,60,19,1415,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0
2,9014,A. Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,180,80,FC Bayern München,German 1. Bundesliga,1.0,SUB,Netherlands,Left,2,4,5,High/Low,Normal (170-185),"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Diver, Injury Prone, Avoids Using Weaker Foot,...",93.0,86.0,83.0,92.0,32.0,64.0,80,85,50,86,86,93,85,83,76,90,93,93,93,89,91,86,61,78,65,90,47,39,89,84,80,84.0,29,26,26,10,8,11,5,15,NaN,87,87,87,90,90,90,90,90,90,90,90,90,81,81,81,90,67,67,67,67,67,58,49,49,49,58,17,1415,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0
3,41236,Z. Ibrahimović,ST,90,90,52500000.0,275000.0,32,195,95,Paris Saint-Germain,French Ligue 1,1.0,ST,Sweden,Right,4,4,5,Medium/Low,Normal (185+),"#Poacher, #Aer

In [ ]:
#Create binary features for body_type

df.body_type = df.body_type.str.split().str.get(0)
df = pd.concat([df, pd.get_dummies(df.body_type,prefix='body')], axis=1)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
#Create a score for left and right foot, preferred foot will always get a 5, and the opposite foot will have the score of "weak_foot", which has a maximum score of 5

#df.loc[df['preferred_foot'] == 'Right', 'right_foot'] = 5
#df.loc[df['preferred_foot'] == 'Left', 'left_foot'] = 5
#df.loc[df['right_foot'].isna(), 'right_foot'] = df.weak_foot
#df.loc[df['left_foot'].isna(), 'left_foot'] = df.weak_foot
#df.head()

def right_foot_ability(preferred_foot, weak_foot):
    if preferred_foot == "Right":
        right = 5
    if preferred_foot == "Left":
        right = weak_foot
    return right
df['right_foot'] = df.apply(lambda x: right_foot_ability(x.preferred_foot, x.weak_foot), axis=1)

def left_foot_ability(preferred_foot, weak_foot):
    if preferred_foot == "Right":
        left = weak_foot
    if preferred_foot == "Left":
        left = 5
    return left
df['left_foot'] = df.apply(lambda x: left_foot_ability(x.preferred_foot, x.weak_foot), axis=1)

In [ ]:
df[['preferred_foot', 'weak_foot', 'right_foot', 'left_foot']]

,preferred_foot,weak_foot,right_foot,left_foot
0,Left,3,3,5
1,Right,4,5,4
2,Left,2,2,5
3,Right,4,5,4
5,Right,4,5,4
...,...,...,...,...
19234,Right,3,5,3
19235,Right,3,5,3
19236,Right,3,5,3
19237,Right,3,5,3


In [ ]:
#Filling in missing data for league_level (some players are unemployed so there is no league level), mentality_composure, value_eur and wage_eur

#Missing values for league level would be filled in by the maximum value in the column (to indicate that their quality is assumed 
#to be low that no teams are interested in signing them, we assign them to the lowest league level, which is represented by the maximum value)

df[df['league_level'].isna()].head()

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_name,league_level,club_position,nationality_name,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,season,diff,One Club Player,Avoids Using Weaker Foot,Playmaker,Dives Into Tackles,Finesse Shot,Power Free-Kick,Leadership,Power Header,Technical Dribbler,Early Crosser,Takes Finesse Free Kicks,Through Ball,Giant Throw-in,Beat Offside Trap,Outside Foot Shot,Long Passer,Set Play Specialist,Chip Shot,Diver,Team Player,Injury Free,Injury Prone,Swerve Pass,Solid Player,Selfish,Speed Dribbler,Flair,Long Shot Taker,Long Throw-in,Backs Into Player,Target Forward,Speedster,Complete Defender,Dribbler,Tackling,Acrobat,Poacher,Crosser,FK Specialist,Complete Forward,Complete Midfielder,Engine,Clinical Finisher,Aerial Threat,Tactician,Distance Shooter,Strength,att_workrate,def_workrate,body_Lean,body_Normal,body_Stocky,body_Unique,right_foot,left_foot
151,209119,F. Amorebielsa,"CB, LB",82,82,NaN,NaN,28,194,83,NaN,NaN,NaN,NaN,Venezuela,Left,3,2,3,Medium/High,Lean,"#Aerial Threat, #Strength","Injury Prone, Dives Into Tackles (AI)",53.0,47.0,56.0,50.0,81.0,85.0,48,38,79,67,53,44,41,34,50,60,53,53,44,73,43,69,68,68,91,41,93,77,49,59,56,76.0,81,84,83,15,5,12,6,12,NaN,57,57,57,54,56,56,56,54,57,57,57,56,62,62,62,56,69,73,73,73,69,75,82,82,82,75,15,1415,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,1,0,0,0,3,5
249,178007,Miguel Veloso,"CDM, CM",80,80,NaN,NaN,28,180,78,NaN,NaN,NaN,NaN,Portugal,Left,4,2,3,Medium/Medium,Normal,Null,"Long Passer (AI), Long Shot Taker (AI)",68.0,72.0,82.0,78.0,75.0,74.0,76,64,66,83,69,76,85,83,85,83,69,67,73,78,75,82,74,75,76,83,70,83,70,83,72,74.0,70,78,75,15,6,8,7,14,NaN,74,74,74,77,77,77,77,77,79,79,79,79,80,80,80,79,78,80,80,80,78,77,76,76,76,77,16,1415,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,1,0,0,4,5
289,209097,O. Cardosa,ST,80,80,NaN,NaN,30,188,84,NaN,NaN,NaN,NaN,Paraguay,Left,2,2,3,Medium/Medium,Normal,"#Poacher, #Distance Shooter, #FK Specialist, #...","Power Free-Kick, Avoids Using Weaker Foot, Lon...",42.0,87.0,72.0,69.0,36.0,71.0,67,87,83,71,80,73,80,87,71,72,32,51,50,79,31,90,34,53,84,86,72,29,87,75,84,74.0,30,32,34,12,7,6,14,11,NaN,80,80,80,71,79,79,79,71,76,76,76,69,69,69,69,69,54,59,59,59,54,52,53,53,53,52,16,1415,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0,1,0,0,2,5
406,20209,Ricardo Costa,"CB, RB",78,78,NaN,NaN,33,183,80,NaN,NaN,NaN,NaN,Portugal,Right,3,2,2,Low/Medium,Stocky,Null,Dives Into Tackles (AI),54.0,37.0,44.0,52.0,77.0,83.0,40,24,87,52,34,47,24,27,46,54,52,55,58,71,65,69,90,74,85,31,89,77,55,44,39,72.0,75,74,85,13,14,6,7,12,NaN,53,53,53,49,5

In [ ]:
df['league_level'] = df['league_level'].fillna(df['league_level'].max())
df[df['league_level'].isna()].head()

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_name,league_level,club_position,nationality_name,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,season,diff,One Club Player,Avoids Using Weaker Foot,Playmaker,Dives Into Tackles,Finesse Shot,Power Free-Kick,Leadership,Power Header,Technical Dribbler,Early Crosser,Takes Finesse Free Kicks,Through Ball,Giant Throw-in,Beat Offside Trap,Outside Foot Shot,Long Passer,Set Play Specialist,Chip Shot,Diver,Team Player,Injury Free,Injury Prone,Swerve Pass,Solid Player,Selfish,Speed Dribbler,Flair,Long Shot Taker,Long Throw-in,Backs Into Player,Target Forward,Speedster,Complete Defender,Dribbler,Tackling,Acrobat,Poacher,Crosser,FK Specialist,Complete Forward,Complete Midfielder,Engine,Clinical Finisher,Aerial Threat,Tactician,Distance Shooter,Strength,att_workrate,def_workrate,body_Lean,body_Normal,body_Stocky,body_Unique,right_foot,left_foot


In [ ]:
#Some players in season 14-15 and 15-16 did not have the value in mentality_composure, we compute the mean differences between 
#mentality_composure and overall rating and apply that to fill in the missing values

df[df['mentality_composure'].isna()]['season'].value_counts()  #should not see anything output if all values have been filled in

#Apply when mentality_composure were filled in by bfill in the previous session

#df['diff'] = df.overall - df['mentality_composure']
#df['mentality_composure'] = df['mentality_composure'].fillna(df['overall']-round(df['diff'].mean()))

Series([], Name: season, dtype: int64)

In [ ]:
#Fill the missing values in value_eur and wage_eur as 0, to reflect the fact that some players are unemployed

df['value_eur'] = df['value_eur'].fillna(0)
df['wage_eur'] = df['wage_eur'].fillna(0)
df[df['short_name']=="D. Forlán"]

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_name,league_level,club_position,nationality_name,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,season,diff,One Club Player,Avoids Using Weaker Foot,Playmaker,Dives Into Tackles,Finesse Shot,Power Free-Kick,Leadership,Power Header,Technical Dribbler,Early Crosser,Takes Finesse Free Kicks,Through Ball,Giant Throw-in,Beat Offside Trap,Outside Foot Shot,Long Passer,Set Play Specialist,Chip Shot,Diver,Team Player,Injury Free,Injury Prone,Swerve Pass,Solid Player,Selfish,Speed Dribbler,Flair,Long Shot Taker,Long Throw-in,Backs Into Player,Target Forward,Speedster,Complete Defender,Dribbler,Tackling,Acrobat,Poacher,Crosser,FK Specialist,Complete Forward,Complete Midfielder,Engine,Clinical Finisher,Aerial Threat,Tactician,Distance Shooter,Strength,att_workrate,def_workrate,body_Lean,body_Normal,body_Stocky,body_Unique,right_foot,left_foot
409,49072,D. Forlán,ST,78,78,0.0,0.0,35,180,78,NaN,NaN,5.0,NaN,Uruguay,Right,5,3,3,High/Medium,Normal,Null,"Beat Offside Trap, Long Shot Taker (AI), Outsi...",66.0,81.0,79.0,80.0,28.0,66.0,77,79,64,80,85,80,87,80,73,86,63,68,72,73,69,85,76,60,74,80,49,23,81,80,81,72.0,22,25,25,8,12,9,5,8,NaN,78,78,78,78,78,78,78,78,78,78,78,78,71,71,71,78,56,58,58,58,56,49,46,46,46,49,13,1415,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0,1,0,0,5,5


In [ ]:
#Drop player_tags because less than 10% of players have tags, and they are sufficiently reflected in the ratings in numbers
#For player_traits, instead of creating one column for every single traits as a binary feature, we categorize them into several categories
#and calculate a score by considering whether the trait is positive or negative

print('Players with player_tags:',len(df[df['player_tags']!="Null"]))
print('Players with player_traits:',len(df[df['player_traits']!="Null"]))
print('Total number of players:',len(df))

Players with player_tags: 11305
Players with player_traits: 57819
Total number of players: 126288


In [ ]:
def injury_risk_calulation(injury_free, solid_player, injury_prone):
    injury_risk = 0
    if injury_free == 1:
        injury_risk = -2
    if solid_player == 1:
        injury_risk = -1
    if injury_prone == 1:
        injury_risk = 1
    return injury_risk
df['injury_risk'] = df.apply(lambda x: injury_risk_calulation(x['Injury Free'], x['Solid Player'], x['Injury Prone']), axis=1)

display(df[df['sofifa_id'] == 173731][['short_name','sofifa_id','Injury Free','Solid Player','Injury Prone','injury_risk']])
#display(df[df['short_name'] == 'G. Bale'][['short_name','sofifa_id','Injury Free','Solid Player','Injury Prone','injury_risk']])

display(df[df['sofifa_id'] == 189596][['short_name','sofifa_id','Injury Free','Solid Player','Injury Prone','injury_risk']])
#display(df[df['short_name'] == 'T. Müller'][['short_name','sofifa_id','Injury Free','Solid Player','Injury Prone','injury_risk']])

,short_name,sofifa_id,Injury Free,Solid Player,Injury Prone,injury_risk
16,G. Bale,173731,0.0,0.0,0.0,0
17,G. Bale,173731,0.0,0.0,0.0,0
6,G. Bale,173731,0.0,0.0,1.0,1
14,G. Bale,173731,0.0,0.0,1.0,1
31,G. Bale,173731,0.0,0.0,1.0,1
77,G. Bale,173731,0.0,0.0,1.0,1
148,G. Bale,173731,0.0,0.0,1.0,1
204,G. Bale,173731,0.0,0.0,1.0,1


,short_name,sofifa_id,Injury Free,Solid Player,Injury Prone,injury_risk
28,T. Müller,189596,0.0,0.0,0.0,0
34,T. Müller,189596,1.0,0.0,0.0,-2
42,T. Müller,189596,1.0,0.0,0.0,-2
62,T. Müller,189596,1.0,0.0,0.0,-2
68,T. Müller,189596,1.0,0.0,0.0,-2
62,T. Müller,189596,0.0,1.0,0.0,-1
55,T. Müller,189596,0.0,1.0,0.0,-1
35,T. Müller,189596,0.0,1.0,0.0,-1


In [ ]:
def teamwork_calulation(leadership, team_player, selfish):
    teamwork = 0
    if leadership == 1:
        teamwork += 1
    if team_player == 1:
        teamwork += 1
    if selfish == 1:
        teamwork -= 1
    return teamwork
df['teamwork'] = df.apply(lambda x: teamwork_calulation(x['Leadership'], x['Team Player'], x['Selfish']), axis=1)

df[df['sofifa_id'] == 183711][['short_name','sofifa_id','Leadership','Team Player','Selfish','teamwork']]
#df[df['short_name'] == 'J. Henderson'][['short_name','sofifa_id','Leadership','Team Player','Selfish','teamwork']]

,short_name,sofifa_id,Leadership,Team Player,Selfish,teamwork
484,J. Henderson,183711,0.0,0.0,0.0,0
470,J. Henderson,183711,1.0,1.0,0.0,2
317,J. Henderson,183711,1.0,1.0,0.0,2
234,J. Henderson,183711,1.0,1.0,0.0,2
240,J. Henderson,183711,1.0,1.0,0.0,2
176,J. Henderson,183711,1.0,1.0,0.0,2
53,J. Henderson,183711,1.0,1.0,0.0,2
102,J. Henderson,183711,1.0,1.0,0.0,2


In [ ]:
def passing_traits_calulation(swerve_pass, long_passer, through_ball, early_crosser, playmaker, set_play_specialist):
    passing_traits = 0
    if swerve_pass == 1:
        passing_traits += 1
    if long_passer == 1:
        passing_traits += 1
    if through_ball == 1:
        passing_traits += 1
    if early_crosser == 1:
        passing_traits += 1
    if playmaker == 1:
        passing_traits += 1
    if set_play_specialist == 1:
        passing_traits += 1
    return passing_traits
df['passing_traits'] = df.apply(lambda x: passing_traits_calulation(x['Swerve Pass'], x['Long Passer'], x['Through Ball'],
                                                                   x['Early Crosser'], x['Playmaker'], x['Set Play Specialist']), axis=1)

df[df['sofifa_id'] == 177003][['short_name','sofifa_id','Swerve Pass','Long Passer','Through Ball','Early Crosser','Playmaker','Set Play Specialist','passing_traits']]
#df[df['short_name'] == 'L. Modrić'][['short_name','sofifa_id','Swerve Pass','Long Passer','Through Ball','Early Crosser','Playmaker','Set Play Specialist','passing_traits']]

,short_name,sofifa_id,Swerve Pass,Long Passer,Through Ball,Early Crosser,Playmaker,Set Play Specialist,passing_traits
17,L. Modrić,177003,0.0,1.0,0.0,0.0,1.0,0.0,2
19,L. Modrić,177003,0.0,1.0,0.0,0.0,1.0,0.0,2
14,L. Modrić,177003,0.0,1.0,0.0,0.0,1.0,0.0,2
15,L. Modrić,177003,0.0,1.0,0.0,0.0,1.0,0.0,2
5,L. Modrić,177003,0.0,1.0,0.0,0.0,1.0,0.0,2
6,L. Modrić,177003,0.0,0.0,0.0,0.0,1.0,0.0,1
31,L. Modrić,177003,0.0,0.0,0.0,0.0,1.0,0.0,1
32,L. Modrić,177003,0.0,0.0,0.0,0.0,1.0,0.0,1


In [ ]:
def attacking_traits_calulation(long_shot_taker, finesse_shot, chip_shot, outside_foot_shot, takes_finesse_free_kicks, beat_offside_trap):
    passing_traits = 0
    if long_shot_taker == 1:
        attacking_traits += 1
    if finesse_shot == 1:
        attacking_traits += 1
    if chip_shot == 1:
        attacking_traits += 1
    if outside_foot_shot == 1:
        attacking_traits += 1
    if takes_finesse_free_kicks == 1:
        attacking_traits += 1
    if beat_offside_trap == 1:
        attacking_traits += 1
    return attacking_traits
df['attacking_traits'] = df.apply(lambda x: passing_traits_calulation(x['Long Shot Taker'], x['Finesse Shot'], x['Chip Shot'],
                                                                   x['Outside Foot Shot'], x['Takes Finesse Free Kicks'], x['Beat Offside Trap']), axis=1)


df[df['sofifa_id'] == 158023][['short_name','sofifa_id','Long Shot Taker','Finesse Shot','Chip Shot','Outside Foot Shot','Takes Finesse Free Kicks','Beat Offside Trap','attacking_traits']]
#df[df['short_name'] == 'L. Messi'][['short_name','sofifa_id','Long Shot Taker','Finesse Shot','Chip Shot','Outside Foot Shot','Takes Finesse Free Kicks','Beat Offside Trap','attacking_traits']]

,short_name,sofifa_id,Long Shot Taker,Finesse Shot,Chip Shot,Outside Foot Shot,Takes Finesse Free Kicks,Beat Offside Trap,attacking_traits
0,L. Messi,158023,0.0,1.0,0.0,0.0,0.0,0.0,1
0,L. Messi,158023,0.0,1.0,0.0,0.0,0.0,0.0,1
1,L. Messi,158023,1.0,1.0,1.0,0.0,0.0,0.0,3
1,L. Messi,158023,1.0,1.0,1.0,0.0,0.0,0.0,3
1,L. Messi,158023,1.0,1.0,1.0,0.0,0.0,0.0,3
0,L. Messi,158023,1.0,1.0,1.0,1.0,0.0,0.0,4
0,L. Messi,158023,1.0,1.0,1.0,1.0,0.0,0.0,4
0,L. Messi,158023,1.0,1.0,1.0,1.0,0.0,0.0,4


In [ ]:
def dribbling_traits_calulation(technical_dribbler, speed_dribbler):
    dribbling_traits = 0
    if technical_dribbler == 1:
        dribbling_traits += 1
    if speed_dribbler == 1:
        dribbling_traits += 1
    return dribbling_traits
df['dribbling_traits'] = df.apply(lambda x: dribbling_traits_calulation(x['Technical Dribbler'], x['Speed Dribbler']), axis=1)


#df[df['sofifa_id'] == 20801][['short_name','sofifa_id','Technical Dribbler','Speed Dribbler','dribbling_traits']]
df[df['short_name'] == 'Cristiano Ronaldo'][['short_name','sofifa_id','Technical Dribbler','Speed Dribbler','dribbling_traits']]

,short_name,sofifa_id,Technical Dribbler,Speed Dribbler,dribbling_traits
1,Cristiano Ronaldo,20801,0.0,1.0,1
1,Cristiano Ronaldo,20801,0.0,1.0,1
0,Cristiano Ronaldo,20801,0.0,1.0,1
0,Cristiano Ronaldo,20801,0.0,1.0,1
0,Cristiano Ronaldo,20801,0.0,1.0,1
1,Cristiano Ronaldo,20801,0.0,1.0,1
1,Cristiano Ronaldo,20801,0.0,1.0,1
2,Cristiano Ronaldo,20801,0.0,1.0,1


In [ ]:
def defending_traits_calulation(power_header, dives_into_tackles):
    defending_traits = 0
    if power_header == 1:
        defending_traits += 1
    if dives_into_tackles == 1:
        defending_traits += 1
    return defending_traits
df['defending_traits'] = df.apply(lambda x: defending_traits_calulation(x['Power Header'], x['Dives Into Tackles']), axis=1)

#df[df['sofifa_id'] == 164240][['short_name','sofifa_id','Power Header','Dives Into Tackles','defending_traits']]
df[df['short_name'] == 'Thiago Silva'][['short_name','sofifa_id','Power Header','Dives Into Tackles','defending_traits']]

,short_name,sofifa_id,Power Header,Dives Into Tackles,defending_traits
14,Thiago Silva,164240,1.0,0.0,1
8,Thiago Silva,164240,1.0,0.0,1
12,Thiago Silva,164240,1.0,0.0,1
21,Thiago Silva,164240,1.0,0.0,1
28,Thiago Silva,164240,1.0,0.0,1
41,Thiago Silva,164240,1.0,0.0,1
62,Thiago Silva,164240,1.0,0.0,1
71,Thiago Silva,164240,1.0,0.0,1


In [ ]:
print(len(df.columns))
print(list(df.columns))

154
['sofifa_id', 'short_name', 'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'height_cm', 'weight_kg', 'club_name', 'league_name', 'league_level', 'club_position', 'nationality_name', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'body_type', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure', 'defending_marking

In [ ]:
#### Check columns that have NaN
for col in df.columns:
    if df[col].isnull().values.any():
        print (col)

club_name
league_name
club_position
goalkeeping_speed
diff


In [ ]:
df.to_csv('Cleaned_Data/player_cleaned_encoded.csv')

In [ ]:
!zip Cleaned_Data/player_cleaned_encoded.csv.zip Cleaned_Data/player_cleaned_encoded.csv

updating: Cleaned_Data/player_cleaned_encoded.csv (deflated 79%)


In [ ]:
!du -sm Cleaned_Data/player_cleaned_encoded.csv.zip

16	Cleaned_Data/player_cleaned_encoded.csv.zip


**Changes in data**

- Filled in missing values in mentality_composure with by using the mean difference between 'overall' and 'mentality_composure' of all other players
- Filled in missing values in value_eur and wage_eur
- Filled in missing values in league_level for unemployed players
- Created columns with different categories and calculate a score that consider the positive / negative traits of each player (to be normalized before fitting the model)

    - Teamwork
        - Leadership +1
        - Team Player +1
        - No trait 0
        - Selfish -1

    - Injury risk (Mutually exclusive)
        - Injury free -2
        - Solid player -1
        - No trait 0
        - Injury prone +1

    - Passing adv
        - Swerve Pass +1
        - Long Passer +1
        - Through Ball +1
        - Early Cross +1
        - Playmaker +1
        - Set Play Specialist +1

    - Attacking adv
        - Long Shot Taker +1
        - Finesse Shot +1
        - Chip shot +1
        - Outside Foot Shot +1
        - Takes Finesse Free Kicks +1
        - Beat Offside Trap +1

    - Dribbling adv 
        - Technical Dribbler +1
        - Speed Dribbler +1

    - Defending adv
        - Power Header +1
        - Dives Into Tackles +1
       
- Drop the following columns (let's discuss, I am not sure too)
    - All player_tags (because less than 10% of players have tags, and they are sufficiently reflected in the ratings) and player_traits binary columns
    - Body_type column (can use weight and height to reflect)
    - Drop columns of: sofifa_id, short_name, dob, club_team_id, league_name, league_level, club_position, nation related, preferred_foot, weak_foot, body_type, work_rate, player_tags, player_traits, goal keeping related
    - Test by including or excluding the columns: potential, age, value, wage

**Standup Video**

Updates:
    
What did my team work on recently?
- Initial stage of data manipulation, dropped irrelevant columns, converted non-numerical data into numbers of categories, filled in missing values
- Started using Keras Autoencoder in Tensorflow to reduce the number of dimensions to from ~75 to ~20 for each player (as deep learning)
- Use PCA to reduce the number of dimension and compare with the Autoencoder models

https://www.kaggle.com/code/saivarunk/dimensionality-reduction-using-keras-auto-encoder

What is my team working on now?
- Selecting appropriate features (feature engineering) to train the model
- Looking for the appropriate hyperparameters to train the model
- Further steps to create team embedding from player embedding (select 20 players from each team based on overall ratings)

What issues are blocking me?
- Looking for useful metrics to evaluate the embedding after dimensionality reduction
- Exploring the possibility to fit the team embedding into a supervised learning model
- Feature selection (what should be included or excluded as feature representation of each player)